#CO-Following on GitHub
#### MIE1512 \ Jay Patel

## PREFACE

This project is based on a paper: Co-Following on Twitter By Venkata Rama Kiran Garimella and Ingmar Weber in 2014 in whcih they have presented a comprehensive study of co-following relationship in social network users, perticularly on Twitter.

The primary hypothesis is that Two twitter users whose followers have similar other followings (except those two users) are also similar. By using co-following users as feature vectors, the analysis shows the classification on binary preference and similarities in different users based on their follower's behaviour.

Here, We are using the same concept of Co-Following on different social media platform: GitHub.

The notebook is in following order.

1. Introduction
2. Importing Data
3. Data Wrangling
4. Data Preparation
5. Rationale of Features (Classification)
6. Main Analysis (PCA)
7. Interpretation of Results
8. Conclusion
9. Future Possible Work
10. References

## 1. INTRODUCTION
In paper on co-following on Twitter, Garimella and Weber have shown the coprehensive study on co-following relationship in social network based on Twitter users' data. They showed that how this observation contributes to (1) user classification on Twitter (2) eliciting opportunities for computational social science and (3) improving online marketing by identifying cross-selling opportunitues.

They started with classification of user based on co-following data to show how accurately we can predict one user's preference on following one from two organizations. Which also paved the way for the further analysis as the accuracy of classification shows nothing else but significance of co-following data.

After that they have found the cosine similarity among the organizations based on their co-following features and then mapped them into 2D plot with the help of Multidimensional Scaling (MDS).
Their result showed how two or more fundamentally different organizations falls near to each other (i.e. seemed similar) even though they don't share any explicit connection.

In [4]:
from pyspark.sql.types import *
from pyspark.ml.feature import CountVectorizer
from pyspark.ml.feature import PCA
from pyspark.sql.functions import lit
from pyspark.mllib.classification import SVMWithSGD, SVMModel
from pyspark.mllib.regression import LabeledPoint
from pyspark.mllib.linalg import SparseVector
from pyspark.mllib.evaluation import BinaryClassificationMetrics
from pyspark.ml.linalg import Vector as MLVector, Vectors as MLVectors
from pyspark.mllib.linalg import Vector as MLLibVector, Vectors as MLLibVectors
from pyspark.sql import DataFrame

#non-spark library
from functools import reduce
import numpy as np
import matplotlib.pyplot as plt

In [5]:
def vectorizer(dataframe, inputCol, outputCol, vocabSize, minDF):
    '''
    function that converts the data to vectors in N space.
    INPUT - DATAFRAME, INPUT COLUMN (in which you have your list of user-data you want to convert into  Vectors)
            OUTPUT COLUMN (under what name you want to save Vectors in DATAFRAME),
            vocabSie - max size of vectors desired (recommended-2000 max for one cluster),
            minDF - min number of times the data should occur for it to be consider as a vector.
    OUTPUT - new DATAFRAME with added column of vectors
    '''
    
    #vectorizing the co-follower features
    vector = CountVectorizer(inputCol=inputCol, outputCol=outputCol,vocabSize=vocabSize, minDF=minDF)
    model = vector.fit(dataframe)

    df_featureLabel = model.transform(dataframe)
    #df_featureLabel.show(truncate=True)
    
    return df_featureLabel

def getPCA(dataframe, inputCol, Ndimension):
    '''
    Function for Dimension Reduction
    Use PCA - Principal Component Analysis
    INPUT - DATAFRAME with feature vector column.
            Ndimension (number of dimension you want to reduce your data into) (Ndimension = 2 for this project)
            inputCol (name of the input dataframe column with high dimensional data)
    OUTPUT - new dataframe with column 'pcaFeature'
    '''
    pca = PCA(k=Ndimension, inputCol=inputCol, outputCol="pcaFeatures")
    model = pca.fit(dataframe)
    return model.transform(dataframe)

def getLabelPoint(datatframe):
    '''
    Function for creating LabeledPoints
    INPUT - Dataframe with column names: 'label' and 'features'
    OUTPUT - RDD of LabeledPoints
    '''
  
    #creating LabeledPoint vectors from features and label columns 
    #dataframe to RDD
    df_to_rdd = datatframe.rdd.map(lambda y: (y.label, y.features))
    
    #labeledPoints
    labeledVectors = df_to_rdd.map(lambda y: LabeledPoint(y[0], MLLibVectors.fromML(y[1])))
    
    return labeledVectors
  
def Classification(labeledVectors):
    '''
    Function for Classification with SVM
    INPUT - RDD of LabeledPoint
    OUTPUT - [AUC-ROC, AUC-PR] type:list
    '''
    #splitting hte data into train and test 
    splits = labeledVectors.randomSplit([0.6, 0.4], 1234)
    train = splits[0]
    test = splits[1]
    # Build the model
    model = SVMWithSGD.train(train)

    # Evaluating the model on training data
    labelsAndPreds = test.map(lambda p: (p.label, model.predict(p.features)))
    #trainErr = labelsAndPreds.filter(lambda lp: lp[0] != lp[1]).count() / float(labeledVectors.count())
    #print("Training Error = " + str(trainErr))
    predictionAndLabels = test.map(lambda lp: (float(model.predict(lp.features)), lp.label))

    metrics = BinaryClassificationMetrics(predictionAndLabels)

    predictions = model.predict(test)
    #print("Area under PR = %s" % metrics.areaUnderPR)
    #print("Area under ROC = %s" % metrics.areaUnderROC)
    
    return [metrics.areaUnderROC, metrics.areaUnderPR]
  
#combining all three collection of organizations

def unionAll(*dfs):
    '''
    function derived from https://stackoverflow.com/questions/33743978/spark-union-of-multiple-rdds
    takes all the dataframes you want to concate as arguments and produce a giant dtaframe 
    only obvious condition is all the input dataframes must have same number of columns with same name. 
    '''
    return reduce(DataFrame.unionAll, dfs)
  

##2. IMPORTING DATA

#### 2.1 IMPORTING DATASETS AND SAVING INTO LOCAL DIRECTORY

The dataset used here is from GHTorrent which is the largest offline mirror data from GitHub REST API which can be accessible through Google BigQuery interface. The latest SQL dump by the time of the project was from April 2018. The dataset imported here is from GHTorrent April 2018 mySQL dump.

Google account and credentials used for BigQuery: Shaily Patel (Team Member)

In [8]:
%sh wget -P/FileStore/tables https://storage.googleapis.com/157watcher/organizations1.csv

In [9]:
%sh wget -P/FileStore/tables https://storage.googleapis.com/157watcher/followers1.csv

In [10]:
#explore the local directory
dbutils.fs.ls("file:/FileStore/tables/")

####2.2 CREATING SPARK DATAFRAMES
From the imported data in form of CSV, now we convert those data to spark readable format: <i>pyspark.sql.dataframe.DataFrame<i>

We also specify the schema manually as spark can not encode the timestamp in proper datatype from the BigQuery.

In [12]:
#building Schema
organizationsSchema = StructType([StructField('org_id', StringType(), True, 
                                             {'description': 'unique organization id', 'type': 'integer'}), 
                                 StructField('user_id', StringType(), True, 
                                             {'description': 'unique member(user) id', 'type': 'integer'}), 
                                 StructField('created_at', TimestampType(), True, 
                                             {'description': 'when the user added as member', 'type':'timestamp',\
                                              'warning':'the date recorded if GHTorrent has recorded corresponding \
                                              event, otherwise the date will correspond to users or organizations \
                                              joining date.'})])

followersSchema = StructType([StructField('user_id', StringType(), True, 
                                             {'description': 'unique organization id', 'type': 'integer'}), 
                                 StructField('follower_id', StringType(), True, 
                                             {'description': 'unique member(user) id', 'type': 'integer'}), 
                                 StructField('created_at', TimestampType(), True, 
                                             {'description': 'when the user started following the follower_id',\
                                              'type':'timestamp', 'warning':'the date recorded if GHTorrent has \
                                              recorded corresponding event, otherwise the date will correspond \
                                              to users or followers joining date.'})])

In [13]:
organizations = spark.read.csv('file:/FileStore/tables/organizations1.csv',header=True, schema=organizationsSchema)
followers = spark.read.csv('file:/FileStore/tables/followers1.csv', header=True, schema=followersSchema)
organizations.printSchema()
followers.printSchema()

####2.3 Creating temporary table views for running query with magic SQL

In [15]:
organizations.createOrReplaceTempView("organizations")
followers.createOrReplaceTempView("followers")

##3. DATA-WRANGLING

####3.1 Basic Questions to access Structure

<b>Q1. Do all records in the dataset contain the same fields?</b>

Since the Schema is set manually,
1. dataset: organizations, contains (1) org_id -> string (2) user_id -> string (3) created_at -> Date
2. dataset: followers, contains (1) user_id -> string (2) follower_id -> string (3) created_at -> Date

<b>Q2. How can you access the same fields across records? By position? By name?</b>

Since the datasets are CSV type with the apprpriate headers, we can access the same field by position as well as by column name.

<b>Q3. How are the records delimited/separated in the dataset? Do you need sophosticated parsing logic  to separate the records form one another?</b>

Datasets procured from GHTorrent is readily available in form of CSV. So there was no need for parsing the data except to convert IDs to string in schema. Because, integer value might generate wrong graphs while querying. Moreover, the timestamp coversion is stated in section 3.3.1

<b>Q4. How are record fields encoded? Human readable strings? Binary numbers? Hash keys? Compressed? Enumerated codes?</b>

All the records are encoded as integer and timestamp(for created_at field). All the fields are human readable.

<b>Q5. What is the complexity of the encoding? Primitives elements like integers, decimal numbers, short strings and so on? Higher-order elements like key-value sets or array?</b>

The time component on dataset was troublesome in original form. Google BigQuery originally save timestamp in a certain format which is not recognised with spark SQL directly.

open issue on github: https://github.com/GoogleCloudPlatform/google-cloud-go/issues/942

However, there is a BigQuery function, brought to attention by group member <i>Mohammed Bubshait</i> convert the datetime format to spark recognizable format. For example, to acquire the organization table, the following query was required:

```sql
SELECT  
  org_id,
  user_id,  
  STRFTIME_UTC_USEC(created_at, '%Y-%m-%d %H:%M:%S') AS created_at  
FROM [ghtorrent-bq:ght_2018_04_01.organization_members]
```


<b>Q7. What are the reletionship types between records and the record fields? Singular (record should have one and only one value for a field, like customer date of birth)? Set-based (record could have many values for the field, like customer shipping addresses)?</b>

Most of the data in both dataset shows one to many relation between `org_id` to `user_id` (as there could be more than one member for an onrganization) and between `user_id` to `follower_id` (as a user could have followed more than one user) in `organizations` and `followers` respectively. Datasets represent these relationship as flat key-value pairs.

####3.2 Basic Questions to assess Data granularity

<b>3.2.1. What kind of thing (person, object, relationship, event, etc.) do the records represent?</b>

In [21]:
%sql
/* 2.1 DATASET - organizations*/
SELECT * FROM organizations

In [22]:
%sql
/* 2.1 DATASET - followers */
SELECT * FROM followers

In [23]:
%sql
desc organizations

In [24]:
%sql
desc followers

The dataset `organizations` provides the unique ID of organization `org_id`, User ID of members of organizations `user_id` and the timestamp `created_at` of when user is added to the organization as member `created_at`.

The dataset `followers` provides the user ID `user_id` and users' ID who is followed by user in first column `follower_id` and the timestamp `created_at` of when user under `user_id` has started following user under `follower_id`.

<b>3.2.2. Are the records homogeneous (represent the same kinds of things)? Or heterogeneous?</b>

Both datasets consist of Homogenenous records of being either strings ( `user_id`, `follower_id`, `org_id` ) or timestamp ( `created_at` ) in every column.

<b>3.2.3. What alternative interpretations of the records are there? For example, if the records appear to be customers, could they actually be all known contacts (only some of which are customers)?</b>

Datasets used for the project has very less dimensions and there is no significant alternative interpretation. However, for dataset `followers`, one can easily interpret inverse that users in `follower_id` has followers in `user_id`.

####3.3 Basic Questions to Access Data Accuracy

<b>3.3.1. Accuracy regarding datatypes </b>

Both dataset `organizations` and `followers` consist of organizations IDs and user IDs. Although github treats both organization and user differently in terms of social features, the ID given to every account (either organization or user) is on same serial.
GHTorrent also treat users and organizations with same serial of IDs. 
REST API provides some attributes such as `type` which differentiate those two from each other. GHTorrent's `user` table provide similar attribute.
Overall, all the ID, whether it is organization or user, both are unique big integers which are saved in table and converted to string while defining schema for easy of vectorizing in later part.

The time component on dataset was troublesome in original form. Google BigQuery originally save timestamp in a certain format which is not recognised with spark SQL directly. For more detail, see section 3.1 Q5

Secondly, the data is machine generated and stored which means there is no question of human error in saving the data.

<b>3.3.2 Anomalies in Dataset</b>

In [31]:
%sql
SELECT count(*), MONTH(created_at) FROM organizations GROUP BY MONTH(created_at) ORDER BY MONTH(created_at)

Monthly detection of membership-event is not very evenly distributed. This could be due to the fact that Organizations can toggle the privacy settings for allows everyone to see the members.

In [33]:
%sql
SELECT count(*), MONTH(created_at) FROM organizations WHERE YEAR(created_at) < 2014 GROUP BY MONTH(created_at) ORDER BY MONTH(created_at)

Above query on `organizations` dataset shows that before 2014, the event are totally missing from month 2,3,4,5,6,7  and August showed very abnormal grown in event detection.

In [35]:
%sql
SELECT count(*), MONTH(created_at) FROM followers GROUP BY MONTH(created_at)

Monthly detection of follow-event shows uniform distribution throughout.

In [37]:
%sql
SELECT count(*), created_at FROM followers  GROUP BY created_at having count(*) > 100000

Above query on `followers` dataset shows that only 5 perticular timestamps have recorded abnormal growth of more than 100000 follow-event throughout the dataset. Which might be the missing events in GHTorrent which were then replaced by the creation date of user or follower, whichever is the latest.

#### 3.4 Basic Questions to Access Data Temporality

<b>3.4.1. When was the dataset collected?</b>

Dataset imported from the Google BigQuery is SQLdump of GHTorrent dated: `1st April, 2018`.

In [41]:
%sql
/*1.1 DATASET - organizations*/
SELECT * FROM organizations ORDER BY created_at DESC

In [42]:
%sql
/*1.1 DATASET - followers*/
SELECT * FROM followers ORDER BY created_at DESC

Result shows that latest dataset is from the date 30th March, 2018. Which also supported by the dataset name in Google BigQuery as `ght_2018_04_01`

<b>3.4.2 Were all the records and record fields collected/measured at the same time? If not, is the temporal range significant?</b>

>The `created_at` field is only filled in accurately for memberships for which GHTorrent has recorded a corresponding event. Otherwise, it is filled in with the latest date that the corresponding user or organization has been created. <i>source: http://ghtorrent.org/relational.html</i>

In short, data is recorded as happened in real-time as best as possible.

In [45]:
%sql
SELECT created_at, count(*) FROM organizations GROUP BY created_at HAVING count(*) > 200 

Above query on `organizations` shows the ONLY THREE days and perticular TIME when more than 200 membership-event has been recorded. Which can be supported as above quoted GHTorrent's method of providing creation date of organizations or member instead of original membership-event timestamp in case of missing events.

<b>3.4.3 Are the timestamps associated with collection of the data known and available (as a record field) or as associated metadata? </b>

There is no metadata available on when the data is collected. But, each dump has the end date of the data collection.

<b>3.4.4 Have some records or record field values been modified after the time of creation? Are the timestamps of these modification available?</b>

For the dataset `organizations`, time of organization and any user get together in organization-member connection is recorded as that time of event. However, there could be some inaccuracy with the idnividual request to the data from REST API which is confirmed by Gousios as he has observed that some REST API calls return slightly modified results if htey queried in different time moments.[2] 

Secondly, an organization has control over whether to display members publicaly or not to. Therefore, the data we get is only from those organizations which has given the access to.

In the dataset `followers`, again, the data is recorded as it happened in time. However, there are some possibility of data inconsistency such as: user account get deleted, user stop following previously followed user, etc.

<b>3.4.5 In what ways can you determine if the data is "stale"? </b>

We can use the GitHub REST API manually via python library such as pygithub to check whether the `org_id` in sample corresponds to an active organization as well as whether a `user_id` is been deleted or diactivated currently. We can query random sample using `RAND()` function and generate a list of IDs to check.

#### 3.5 Baic Questions to Access Data Scope

<b>3.5.1 Given the granularity of the dataset, what characteristics of the things represented by the records are captured by the record fields? What characteristics are not captured? </b>

The dataset, or perticularly, the tables from GHTorrent datasets: `organizations` and `followers` are very shallow in terms of granularity since both datasets have the detail such as: the organizations and their memebr users and event-time of user being member of organization; the user and followings and event-time of user started following other. There is no details such as user status in organization or total contribution in repositories owned by organizations.

<b>3.5.2 Are the record fields consistent? </b>

<b>3.5.3 Do the records in the dataset represent the entire population of the associated things?</b>

Since the GHTorrent have made it clear that the dataset has only points (i.e. `org_id` and `user_id`) which has information publically available and GHTorrent do not capture (even as `null` point) those `org_id` or `user_id` which have privacy restrictions.

<b>3.5.4 Are there missing racords? Is it randomly missing or systematically missing?</b>

GHTorrent declares that the event of being memebr of an organization and user following other user has been captured as they occur. However, due to malfunctioning in the mirroring system (software or network) could result in some part of data that are missing. This missing event data can be restored if it is still in newest 300 events per repository(this limit is imposed by GitHub REST API). Any most frequently changing project could have more than 300 events per day which means the missing events of those repositories can not be restored.[2]

##4. Co-Following on GitHub

####4.1 VARIATION OF CO-FOLLOWING ON GITHUB FOR ORGANIZATIONS
GitHub is fundamentaly different from the Twitter. In twitter, there is no difference between an individual user such as @tim_cook and an organization such as @Apple. These both are similar in terms of their behavious in following and followed by others However, GitHub has explicit difference between a user and an organization. In GitHub, there could be a user who can follow other users and followed by other users as well but an organization can not follow other users or organizations and vise versa. Organization in GitHub can have members(which are individual users) and these members has to be invited by the organization itself for membership. One user could be member of several organizations.

Since GitHub is a social media for collaborative work especially in the field of software engineering, there are hardly any individuals who has solely gained popularity. It is always a combine effort from a group of people (random group of contributors for a pertiuclar project or members of an organization) for making their project famous on the platform.

Here, I am proposing two different methods derived from original co-Following method of twitter to look into the similarity among organizations in the GitHub: 

<b>(1) CoMembership : </b> For an organization in GitHub, the members of which could also be member of other organizations which are called CoMemberships of that organization. For example, if a user @dev is one of the member of organization @team and @dev also a member of other organizations such as @org1, @org2, and @org3; then @org1, @org2, and @org3 are coMemberships of @team with respect to @dev.

<b>(2) CoFollowing : </b>As discussed earlier, organizations don't have privilage to follow other organizations or users, but their members have certain followship as they are individual account. So coFollowing of an organizations are the users' account which are followerd by the members of that organizations. For example, if user @dev follows other users such as @user1, @user2, and @user3; then @user1, @user2, and @user3 are called coFollowings of the organization @team with respect to @dev. Note that here we are assuming that being member of an organization is similar to follow it to get similar model as that of twitter.


<b>NOTE</b> After November 2015, Organizations can select whether membership information is revealed to external parties. That means we are only analysing the organizations which have not made membership information private.

####4.2 Example of Data

Table 1 shows the organizations ID in first column and their members' ID in second column. It is one to many relationship since one organization could have more than one users as member.
For example, Table 1 shows organization <i>A</i> has user <i>a,b,c,d,e</i> as members. Note that we use the word <em>root organization</em> as organizations on which we are performing analysis.

Table 1: Root organization and its members

root organization | member 
--- | --- 
A | a
A | b
A | c
A | d
A | e
B | c
B | d
B | f
C | g
C | t
C | y

Table 2 shows the users and their membership to respective organizations. It is again one to many relationship but reverse from table 1 since one user could be member of multiple organizations simulteneously.
For example, table 2 shows user <i>a</i> has membership in organizations <i>A,F,G</i>, and <i>X</i> with <i>A</i> being root organization. Every user in this table must have at least one root organization.

Table 2: user and its organization

user | organiztion 
--- | --- 
a | A
a | F
a | G
a | X
b | A
b | D
b | F
c | W
c | A
c | B
d | A
d | B
d | R
e | A
f | B
f | H
g | C
g | R
t | C
t | G
t | H

Table 3 shows the user and their followers. It is one to many relationship as one user can follow more than one users.

Table 3: user and their follower

user | follwoer 
--- | --- 
a | q
a | w
a | s
a | z
b | v
b | d
b | f
b | g
c | v
c | f
c | r
d | w
d | e
d | f
e | q
e | a
f | g
f | v
f | e
f | d
g | y
g | h
g | f
t | e
t | r
t | y
y | h
y | g
y | a



#### 4.2.1 co-membership Data

Now we join the organization to organizations on one to many relationship based on the member users of root organizations. The first column in table 4 shows root organizations and second column are all other organization connected to root organization with relationship of co-membership. For example, as seen in above table 1 and 2, root organization <i>C</i> has three members: <i>g,t</i>, and <i>y</i> who are also members of organizations <i>C,R,G,H</i> cummulatively which we called as co-membership of root organization. So <i>R,H,G</i> are co-membership of root organization <i>C</i>. Note that we remove <i>C</i> from co-membership as it is obvious that organiztions can not be counted as its own co-member. 

Table 4: Root Organization and their co-members

root organization | co-member 
--- | --- 
A | F
A | G
A | X
A | D
A | W
A | B
A | R
B | A
B | W
B | R
B | H
C | R
C | G
C | H


Finally, table 5 shows grouped co-members of every root organization.

Table 5: Root organization and their grouped co-members

root organization | co-member
--- | ---
A | [F, G, X, D, W, B, R]
B | [A, W, R, H]
C | [R, G, H]


#### 4.2.2 co-following Data

Now we join the organization to users on one to many relationship based on the member users of root organizations. The first column in table 6 shows root organizations and second column are all followers of the members of root organization. For example, as seen in above table 1 and 3, root organization <i>C</i> has three members: <i>g,t</i>, and <i>y</i> who follows <i>y,h,f,e,r,g</i>, and <i>a</i> cummulatively which we called as co-followings of root organization. So <i>y,h,f,e,r,g</i>, and <i>a</i> are co-membership of root organization <i>C</i>. Note that we <u>do not</u> remove <i>y</i> from co-following here as opposed to co-membership as <i>y</i> is coming from the following of <i>g</i> and <i>t</i> which is fair. 

Table 6: Root Organization and their co-members

root organization | co-following 
--- | --- 
A | q
A | w
A | s
A | z
A | v
A | d
A | f
A | g
A | r
A | w
A | e
A | a
B | v
B | f
B | r
B | w
B | e
B | d
C | y
C | h
C | f
C | e
C | r
C | g
C | a


Finally, table 7 shows grouped co-following of every root organization.

Table 4: Root organization and their grouped co-following

root organization | co-following
--- | ---
A | [q, w, s, z, v, d, f, g, r, w, e, a]
B | [v, f, r, w, e, d]
C | [y, h, f, e, r, g, a]

#### 4.2.3 Feature Matrix

We use this co-member as feature vector with countVectorizer from MLlib.
Let's consider all co-member in dataset as n-dimesion vector.
For AllCoMembers = [A, B, D, F, G, H, R, W, X]

So the feature matrix would look like this:

ORG | A | B | D | F | G | H | R | W | X
--- | --- | --- | --- | --- | --- | --- | --- | --- 
A | 0 | 1 | 1 | 1 | 1 | 0 | 1 | 1 | 1
B | 1 | 0 | 0 | 0 | 0 | 1 | 1 | 1 | 0
C | 0 | 0 | 0 | 0 | 1 | 1 | 1 | 0 | 0

Note: Similar structure is obtained for co-follwer analysis.

##BIG QUESTION | do co-membership and co-following really matters?
Before we start using co-membership and co-following data as feature data for further analysis,  we ask ourselves whether co-membership and co-following data really give significant information about the organizations. To answer this question, we ask another more specific question that, for co-membership data, given the users and which organizations they possess membership of, can we predict if certain user be member of any perticular organization and for co-following data, given users and their followers, can we predict whether a user be member of a certain organization. This will be answered using classification algorithm (SVM) for evaluation.

##5. Rationale of Features (Binary user Classification)

####BIG QUESTION - do co-membership and co-following really matters?
Before we start using co-membership and co-following data as feature data for further analysis,  we ask ourselves whether co-membership and co-following data really give significant information about the organizations. To answer this question, we ask another more specific question that, for co-membership data, given the users and which organizations they possess membership of, can we predict if certain user be member of any perticular organization and for co-following data, given users and their followers, can we predict whether a user be member of a certain organization. This will be answered using classification algorithm (SVM) on organization pairs for evaluation.

In [60]:
%sql
SELECT 
  org_id,
  COUNT(user_id) as totalMember
FROM 
  organizations
GROUP BY org_id
ORDER BY totalMember DESC
LIMIT 10

#### 5.1 Steps for Labeled Data
1. Select two organizations
2. Get two dataframe, one for each organization as `org_id` in first column and  `coMembership` in second.
3. Group all co-member on organization as described above.
4. Add one more column in both dataframes that act as label: 1 for dataframe1 and 0 for dataframe2 <strong>*(see `for loop` in code below for data processing decribed in step 1 to 4)*</strong>
5. Use columns `labels` and `features` as input for creating RDD of LabeledPoints. <strong>*(see function `getLabelPoint`)*</strong>

#### 5.2 Steps for finding Classification accuracy

1. Input - labeledData for co-membership data ** *(from above mentioned steps)* **
2. **For** every-possible pair of root organizations **do** Binary Classification 
3. Evaluate accuracy for each pair
4. Find average accuaracy

same steps repeated for coFollowing data as well.

*NOTE: following code takes pair of organizations and do classification over co-following data as well as co-membership data(another loop). The loop is doing data preprocessing initially and then use functions such as vectorizer, classification, etc. which defined above. However, the loop has very long runtime so here we just select 3 organizations (with nearly 600 members in each) to get classification result for each pair (total 2^3 = 8 pairs) which takes 3 minutes(8 times 3 = 24 minutes for each loop which is approximately 1 hr for both loops) of time. I delibarately choose smaller list of organization to run through quickly. As mentioned earlier, classification is just to make sure that co-following and co-membership data really have significance over organizations characteristics. We also limits the vector size to 1000 and minDF=2 *

*NOTE: in PCA, every feature vector is at size of 2000 as the partition value for single cluster in Databricks spark is 2000 and for vector size more than 2000 will exceed the java heap memory and cause error*

Cell below shows the code for creating arbitary list of organizations (here 3 organizations with each having number of members approximately 600)
`for loop` used to make pairs of organzations for classification afterward.

In [63]:
top5_org = [259105, 1007812, 7022203] #selected from top organization query as they have nearly 600 members in each.
pair_of_orgs = []
for i in top5_org:
  for j in top5_org:
    if i != j:
      if (j,i) not in pair_of_orgs:
          pair_of_orgs.append((i,j))

####5.3 Classification on Co-members
Cells below show the `for loop` that measure classification accuracy for every possible pairs of given organizations and at the end gives average accuracy.

In [65]:
accuracy_of_coMembership = []

for num, i in enumerate(pair_of_orgs):
  
    #creating 2 dataframe each for 2 organizations
    #sql query for acquiring co-member data for given organizations
    df1 = spark.sql('''SELECT M.org_id as org, M.user_id as member, N.org_id as coMember FROM
                      (SELECT org_id, user_id FROM organizations WHERE org_id = {0}) AS M JOIN
                      organizations AS N ON M.user_id = N.user_id
                      WHERE N.org_id != {1}'''.format(i[0],i[0]))
    df2 = spark.sql('''SELECT M.org_id as org, M.user_id as member, N.org_id as coMember FROM
                      (SELECT org_id, user_id FROM organizations WHERE org_id = {0}) AS M JOIN
                      organizations AS N ON M.user_id = N.user_id
                      WHERE N.org_id != {1}'''.format(i[1],i[1]))
    
    #dataframe to rdd
    df1_ = df1.rdd.map(lambda p: (p[1], p[2]))
    df2_ = df2.rdd.map(lambda p: (p[1], p[2]))
    
    #convert flat key value list to list of value for each key
    df1tuple__ = df1_.groupByKey().mapValues(list)
    df2tuple__ = df2_.groupByKey().mapValues(list)

    #removing dublicate values if any
    df1tuple = df1tuple__.map(lambda p: (p[0],list(set(p[1]))))
    df2tuple = df2tuple__.map(lambda p: (p[0],list(set(p[1]))))
    
    #removing common members of both dataset 
    df1tuple_ = df1tuple.subtractByKey(df2tuple)
    df2tuple_ = df2tuple.subtractByKey(df1tuple)
    
    #creating spark dataframes with third column added as label with value of 1 and 0 for respective dataframes
    dataframe1 = spark.createDataFrame(df1tuple_, ['member', 'coMember']).withColumn('label', lit(0))
    dataframe2 = spark.createDataFrame(df2tuple_, ['member', 'coMember']).withColumn('label', lit(1))
    
    #join two dataframes
    dataframe = dataframe1.unionAll(dataframe2)
    
    #create new column in dataframe for features vectors
    dataframeVec = vectorizer(dataframe, 'coMember', 'features', 1000, 10)
    
    labelDF = getLabelPoint(dataframeVec)
    
    classification_matrix = Classification(labelDF)
    
    accuracy_of_coMembership.append(classification_matrix[0]) #put 1 for getting average accuracy based on PR, 0 for ROC
    
    print("loop ", num, "complete. Remained ", len(pair_of_orgs) - num)
    
avg_coMember_accuracy = np.mean(accuracy_of_coMembership)
print("The average co-membership classification accuracy is :", avg_coMember_accuracy)

####5.4 Classification on Co-following
Cells below show the `for loop` that measure classification accuracy for every possible pairs of given organizations and at the end gives average accuracy.

In [67]:

accuracy_for_coFollowing = []
for num, i in enumerate(pair_of_orgs):
  
    #creating 2 dataframe each for 2 organizations
    df1 = spark.sql('''SELECT A.org_id, A.user_id, C.follower_id FROM organizations AS A JOIN followers AS C ON A.user_id = C.user_id WHERE A.org_id = {0}'''.format(i[0]))
    df2 = spark.sql('''SELECT A.org_id, A.user_id, C.follower_id FROM organizations AS A JOIN followers AS C ON A.user_id = C.user_id WHERE A.org_id = {0}'''.format(i[1]))
    
    #dataframe to rdd
    df1_ = df1.rdd.map(lambda p: (p[1], p[2]))
    df2_ = df2.rdd.map(lambda p: (p[1], p[2]))
    
    #convert flat key value list to list of value for each key
    df1tuple__ = df1_.groupByKey().mapValues(list)
    df2tuple__ = df2_.groupByKey().mapValues(list)

    #removing dublicate values if any
    df1tuple = df1tuple__.map(lambda p: (p[0],list(set(p[1]))))
    df2tuple = df2tuple__.map(lambda p: (p[0],list(set(p[1]))))

    #removing common members from both dataset 
    df1tuple_ = df1tuple.subtractByKey(df2tuple)
    df2tuple_ = df2tuple.subtractByKey(df1tuple)
    
    #creating spark dataframes with third column added as label with value of 1 and 0 for respective dataframes
    dataframe1 = spark.createDataFrame(df1tuple_, ['org_id', 'follower_id']).withColumn('label', lit(0))
    dataframe2 = spark.createDataFrame(df2tuple_, ['org_id', 'follower_id']).withColumn('label', lit(1))
    
    #join two dataframes
    dataframe = dataframe1.unionAll(dataframe2)
    
    #create new column in dataframe for features vectors
    dataframeVec = vectorizer(dataframe, 'follower_id', 'features', 1000, 2)
    
    #convert feature vector and label column to labeledPoints RDD
    labelDF = getLabelPoint(dataframeVec)
    
    #classification
    classification_matrix = Classification(labelDF)
    
    
    accuracy_for_coFollowing.append(classification_matrix[0]) #put 1 for getting average accuracy based on PR, 0 for ROC

    print("loop ", num, "complete. Remained ", len(pair_of_orgs) - num)
    
avg_cofollowing_accuracy = np.mean(accuracy_for_coFollowing)
print("The average co-membership classification accuracy is :", avg_cofollowing_accuracy)

####5.5 Classification Result Interpretation

##6. Main Analysis (PCA)

####6.1 Procedure for PCA (Principal Component Analysis)
1. Feature vectors for co-membership and co-following are user as N-dimensional data.
2. Implement PCA (from the spark MLlib) to reduce dimensionality of feature vectors

###PCA on Co-membership in Different sets of organizations

####6.2 CoMembership Similarity in Top 10 Organizations

In [72]:
%sql
SELECT 
  org_id,
  COUNT(user_id) as totalMember
FROM 
  organizations
GROUP BY org_id
ORDER BY totalMember DESC
LIMIT 10

In [73]:
%sql
/*To avoid over-complicated query, the organizations IDs are given manually*/
/*this query also makes sure that we dont collect same organizations for its coMembers*/
SELECT M.org_id as org, M.user_id as member, N.org_id as coMember FROM
(SELECT org_id, user_id FROM organizations WHERE org_id IN (3886902, 95143, 3681780, 3815374, 3432832, 2206, 259105, 1007812, 7022203, 2156)) AS M JOIN
organizations AS N ON M.user_id = N.user_id
WHERE N.org_id not in (3886902, 95143, 3681780, 3815374, 3432832, 2206, 259105, 1007812, 7022203, 2156)

##### Description for Code in Below
1. Dataframe generation from sql query
2. creating RDD from dataframe using only column 1(organization id) and 3(co-membership) ** *Note: co-following would be 3rd column in co-following analysis* **
3. Grouped list from Flat-value RDD i.e. from `org_id: comember1, org_id: comember2,...` to `org_id : [comember1, comember2, ...]` 
4. Removing Dublicated from grouped list
5. creating dataframe with two columns: `organizations` and `features`
6. creating feature vectors from `feature` column *(see function **Vectorizer**)*
7. PCA on feature vectors.

**Note: This code get repeated for different set of organizations analysis**

In [75]:
#spark dataframe for above stated query
coMembership_top10 = spark.sql(
'''
SELECT M.org_id as org, M.user_id as member, N.org_id as coMember FROM
(SELECT org_id, user_id FROM organizations WHERE org_id IN (3886902, 95143, 3681780, 3815374, 3432832, 2206, 259105, 1007812, 7022203, 2156)) AS M JOIN
organizations AS N ON M.user_id = N.user_id
WHERE N.org_id not in (3886902, 95143, 3681780, 3815374, 3432832, 2206, 259105, 1007812, 7022203, 2156)
'''
)

#dataframe to RDD in form of tuples like: (organization, coMember)
rdd = coMembership_top10.rdd.map(lambda p: (str(p[0]), str(p[2])))

#convert flat key value list to list of value for each key
groupRDD = rdd.groupByKey().mapValues(list)

#removing dublicate values if any
groupRDD_ = groupRDD.map(lambda p: (p[0],list(set(p[1]))))

#RDD back to Dataframe
DF = spark.createDataFrame(groupRDD_ , ['organizations', 'features'])

#vectorizing the data
#parameter given: Above stated Dataframe, name of input column, name for output column, max number of dimension desired, min number of time a datapoint should appear in organiations to consider as vector
DF2 = vectorizer(DF, 'features', 'vectors', 2000, 5)

#dimension reduction of dataset
DF3 = getPCA(DF2, 'vectors', 2)

####6.2.1 Visualization using Matplotlib

In [77]:
#x and y values of data in list form
X_top10CoMember = DF3.rdd.map(lambda p: p['pcaFeatures'][0]).collect()
Y_top10CoMember = DF3.rdd.map(lambda p: p['pcaFeatures'][1]).collect()

#organization names in a list
top10CoMember = DF3.rdd.map(lambda p: p[0]).collect()

In [78]:
fig, axs = plt.subplots()
plt.title("Co-Membership on Top 10 Organizations")
axs.scatter(X_top10CoMember,Y_top10CoMember)
for i, txt in enumerate(top10CoMember):
    axs.annotate(txt, (X_top10CoMember[i],Y_top10CoMember[i]))
display()

####6.2.2 Plot Interpretation

+ id `3432832` i.e. `fossasia` and id `3681780` i.e. `Azure` are clustured together here. Which shows similarity in those two. Potential reason has been found that,in 2017, Microsoft was GOLD SPONSOR at FOSSASIA Summit. Which was an event for developers, startups and students using <b> open source tools </b> provided by Microsoft.
+ Other than that, there is no significant similarity found in these organizations.

####6.3 CoMembership Similarity in Top Organizations in GHImpact

###GH-IMPACT
While searching for reasonable organizations to compare. I came across to the research work of a PhD student <a href='http://imiller.utsc.utoronto.ca/'>Ian Dennis Miller</a> at University of Toronto. 
He has created a impact measurement of open source project based on the influence level.
>gh-impact measures open source influence. gh-impact is based upon the stars a project receives: an account has a gh-impact score of <b>n</b> if they have <b>n projects</b> with <b>n stars</b>. Higher gh-impact scores correspond to accounts that have many well-used projects. 

<i> source: http://www.gh-impact.com/about/ </i>

<b>top 20 most impactful organizations according to GHImpact score is as below:</b>

name | org_id | gh-impact
--- | --- | ---
google | 95143 | 185
facebook | 2156 | 147
apache | 13369 | 130
Microsoft | 38886902 | 104
mozilla | 1146 | 95
codrops | 636722 | 92
twitter | 5092 | 88
square | 8274| 79
googlesamples | 4438683 | 73
Netflix | 1600 | 72
mapbox | 29218 | 69
spring-projects | 2395257 | 67
thoughtbot | 10963 | 66
github | 142 | 63
angular | 159 | 61
GoogleCloudPlatform | 1007812 | 61
awslabs | 1904672 | 61
yahoo | 7045 | 59
Atom | 1110689 | 59
openstack | 23312 | 58

<i> source: http://www.gh-impact.com/leaderboard/ </i>

The `org_id` is derived from another ghtorrent dataset table called `users` wwhich is not imported here due to its large size.

Here is the query on GoogleBigQuery to produce `org_id` for given username of organization/user. Usernames provided in query are case sensitive. 

```sql
SELECT id, login FROM [ghtorrent-bq:ght_2018_04_01.users] 
WHERE type='ORG' AND login IN ('google','facebook','apache','Microsoft','mozilla'
,'codrops','twitter','square','googlesamples','Netflix','mapbox','spring-projects',
'thoughtbot','github','angular','GoogleCloudPlatform','awslabs','yahoo','Atom',
'openstack')
```

In [82]:
%sql
SELECT M.org_id as org, M.user_id as member, N.org_id as coMember FROM
(SELECT org_id, user_id FROM organizations WHERE org_id IN (7045, 1600, 636722, 5092, 95143, 3886902, 4438683, 13369, 8274, 10963, 23312, 159, 2156, 13369, 8274, 10963, 23312, 159, 2156, 142, 1146, 29218, 1110689, 1007812, 2395257, 1904672)) AS M JOIN
organizations AS N ON M.user_id = N.user_id
WHERE N.org_id not in (7045, 1600, 636722, 5092, 95143, 3886902, 4438683, 13369, 8274, 10963, 23312, 159, 2156, 13369, 8274, 10963, 23312, 159, 2156, 142, 1146, 29218, 1110689, 1007812, 2395257, 1904672)

In [83]:
#spark dataframe for above stated query
coMembership_ghi = spark.sql(
'''
SELECT M.org_id as org, M.user_id as member, N.org_id as coMember FROM
(SELECT org_id, user_id FROM organizations WHERE org_id IN (7045, 1600, 636722, 5092, 95143, 3886902, 4438683, 13369, 8274, 10963, 23312, 159, 2156, 13369, 8274, 10963, 23312, 159, 2156, 142, 1146, 29218, 1110689, 1007812, 2395257, 1904672)) AS M JOIN
organizations AS N ON M.user_id = N.user_id
WHERE N.org_id not in (7045, 1600, 636722, 5092, 95143, 3886902, 4438683, 13369, 8274, 10963, 23312, 159, 2156, 13369, 8274, 10963, 23312, 159, 2156, 142, 1146, 29218, 1110689, 1007812, 2395257, 1904672)
'''
)

#dataframe to RDD in form of tuples like: (organization, coMember)
rdd_ghi = coMembership_ghi.rdd.map(lambda p: (str(p[0]), str(p[2])))

#convert flat key value list to list of value for each key
groupRDD_ghi = rdd_ghi.groupByKey().mapValues(list)

#removing dublicate values if any
groupRDD_ghi_ = groupRDD_ghi.map(lambda p: (p[0],list(set(p[1]))))

#RDD back to Dataframe
DF_ghi = spark.createDataFrame(groupRDD_ghi_ , ['organizations', 'features'])

#vectorizing the dataset
DF2_ghi = vectorizer(DF_ghi, 'features', 'vectors', 2000, 5)

#dimension reduction of dataset
DF3_ghi = getPCA(DF2_ghi, 'vectors', 2)

####6.3.1 Visualization using Matplotlib

In [85]:
#x and y values of data in list form
X_ghi = DF3_ghi.rdd.map(lambda p: p['pcaFeatures'][0]).collect()
Y_ghi = DF3_ghi.rdd.map(lambda p: p['pcaFeatures'][1]).collect()

#organization names in a list
ghiCoMember = DF3_ghi.rdd.map(lambda p: p[0]).collect()

In [86]:
fig2, axs2 = plt.subplots()
plt.title("Co-Membership on Top organizations in GHImpact")
axs2.scatter(X_ghi,Y_ghi)
for i, txt in enumerate(ghiCoMember):
    axs2.annotate(txt, (X_ghi[i],Y_ghi[i]))
fig2.set_size_inches(18.5, 10.5, forward=True)
display()

####6.3.2 Plot Interpretatin
+ `microsoft` (3886902) and `angular` (159) has clustered together. As Microsoft's TypeScript language is the best suitable for the Angular due to following features:
  - Class-based Object Oriented Programming
  - Static Typing
  - Generics
+ `mapbox`(29218), `square` (8274), `spring-project` (2395257), `awslabs` (1904672) and `codrops` (6367722) are clustered together showing similarity in CoMembership.
+ Besides these, there are so many other organizations clustered together which might be the result of exclusiveness in co-membership.

###PCA on Co-following in Different sets of organizations

####6.4 CoFollowing Similarity in Top 10 Organizations

In [90]:
%sql
SELECT 
  A.org_id, B.user_id, C.follower_id
FROM
   (
    SELECT 
      org_id,
      COUNT(user_id) as totalMember
    FROM 
      organizations
    GROUP BY org_id
    ORDER BY totalMember DESC
    LIMIT 10
    ) AS A
JOIN organizations AS B ON B.org_id = A.org_id
JOIN followers AS C ON C.user_id = B.user_id
ORDER BY A.org_id

In [91]:
#spark dataframe for above stated query
CF_top10 = spark.sql(
'''
SELECT 
  A.org_id, B.user_id, C.follower_id
FROM
   (
    SELECT 
      org_id,
      COUNT(user_id) as totalMember
    FROM 
      organizations
    GROUP BY org_id
    ORDER BY totalMember DESC
    LIMIT 10
    ) AS A
JOIN organizations AS B ON B.org_id = A.org_id
JOIN followers AS C ON C.user_id = B.user_id
ORDER BY A.org_id
'''
)

#dataframe to RDD in form of tuples like: (organization, coMember)
rdd_CF_top10 = CF_top10.rdd.map(lambda p: (str(p[0]), str(p[2])))

#convert flat key value list to list of value for each key
groupRDD_CF_top10 = rdd_CF_top10.groupByKey().mapValues(list)

#removing dublicate values if any
groupRDD_CF_top10_ = groupRDD_CF_top10.map(lambda p: (p[0],list(set(p[1]))))

#RDD back to Dataframe
DF_CF_top10 = spark.createDataFrame(groupRDD_CF_top10_ , ['organizations', 'features'])

#vectorizing the dataset
DF2_CF_top10 = vectorizer(DF_CF_top10, 'features', 'vectors', 2000, 5)

#dimension reduction of dataset
DF3_CF_top10 = getPCA(DF2_CF_top10, 'vectors', 2)

#### 6.4.1 Visualization using Matplotlib

In [93]:
#x and y values of data in list form
X_top10CF = DF3_CF_top10.rdd.map(lambda p: p['pcaFeatures'][0]).collect()
Y_top10CF = DF3_CF_top10.rdd.map(lambda p: p['pcaFeatures'][1]).collect()

#organization names in a list
top10CF = DF3_CF_top10.rdd.map(lambda p: p[0]).collect()

In [94]:
fig4, axs4 = plt.subplots()
plt.title("Co-Following on Top 10 Organizations")
axs4.scatter(X_top10CF,Y_top10CF)
for i, txt in enumerate(top10CF):
    axs4.annotate(txt, (X_top10CF[i],Y_top10CF[i]))
fig4.set_size_inches(18.5, 10.5, forward=True)
display()

#### 6.4.2 Plot Interpretation
+ In co-following, `facebook` (2156), `google` (95143), `EpicGames` (3815374) and `github-beta` (7022203) looks similar as they fall very near in the plot.
+ The top10 organizations having very high number of members compare to an average organizations, cause high number of co-following relationship as well. The reduction of this high dimensionality  often not be that accurate and we might loose some important information with noise as well.

#### 6.5 CoFollowing Similarity in Top Organizations in GHImpact

In [97]:
%sql
SELECT 
  A.org_id, A.user_id, C.follower_id
FROM
   organizations AS A JOIN followers AS C ON A.user_id = C.user_id
WHERE
  A.org_id IN (7045, 1600, 636722, 5092, 95143, 3886902, 4438683, 13369, 8274, 10963, 23312, 159, 2156, 13369, 8274, 10963, 23312, 159, 2156, 142, 1146, 29218, 1110689, 1007812, 2395257, 1904672)

In [98]:
#spark dataframe for above stated query
CF_ghi = spark.sql(
'''
SELECT 
  A.org_id, A.user_id, C.follower_id
FROM
   organizations AS A JOIN followers AS C ON A.user_id = C.user_id
WHERE
  A.org_id IN (7045, 1600, 636722, 5092, 95143, 3886902, 4438683, 13369, 8274, 10963, 23312, 159, 2156, 13369, 8274, 10963, 23312, 159, 2156, 142, 1146, 29218, 1110689, 1007812, 2395257, 1904672)
'''
)

#dataframe to RDD in form of tuples like: (organization, coMember)
rdd_CF_ghi = CF_ghi.rdd.map(lambda p: (str(p[0]), str(p[2])))

#convert flat key value list to list of value for each key
groupRDD_CF_ghi = rdd_CF_ghi.groupByKey().mapValues(list)

#removing dublicate values if any
groupRDD_CF_ghi_ = groupRDD_CF_ghi.map(lambda p: (p[0],list(set(p[1]))))

#RDD back to Dataframe
DF_CF_ghi = spark.createDataFrame(groupRDD_CF_ghi_ , ['organizations', 'features'])

#vectorizing the dataset
DF2_CF_ghi = vectorizer(DF_CF_ghi, 'features', 'vectors', 2000, 5)

#dimension reduction of dataset
DF3_CF_ghi = getPCA(DF2_CF_ghi, 'vectors', 2)

#### 6.5.1 Visualization using Matplotlib

In [100]:
#x and y values of data in list form
X_ghiCF = DF3_CF_ghi.rdd.map(lambda p: p['pcaFeatures'][0]).collect()
Y_ghiCF = DF3_CF_ghi.rdd.map(lambda p: p['pcaFeatures'][1]).collect()

#organization names in a list
ghiCF = DF3_CF_ghi.rdd.map(lambda p: p[0]).collect()

In [101]:
fig5, axs5 = plt.subplots()
plt.title("Co-Following on top Organizations in GHImpact")
axs5.scatter(X_ghiCF,Y_ghiCF)
for i, txt in enumerate(ghiCF):
    axs5.annotate(txt, (X_ghiCF[i],Y_ghiCF[i]))
fig5.set_size_inches(18.5, 10.5, forward=True)
display()

####6.5.2 Plot Interpretation
+ `codrops` (636722), `awslabs` (1904672), `googlesample` (4438683) and `spring-projects` (2395257) are clustered together.
+ `netflix` (1600) and `angular` (159) shows more closeness than above stated ones.
+ `mozilla` (5092) and `twitter` (1146) shows similarity.

####6.6 CoFollowing Similarity in Organizations from GitHub Collections

<a href='https://github.com/collections'>GitHub Collections</a> are  curated lists of similar open source projects. GitHub explore an dhand-picked this repositories. Since we are only looking into organizations. We will pickup only those repositories that are owned by an organization rather than a single user.

GitHub collection: <b>Open_Journalism</b> is the collection of repositories made by famous publications and data-driven journalisms to power their newsroom and ensure informatin is reported fairly and accurately.

name | org_id
--- | --- 
voxmedia | 508316 
guardian | 11571 
propublica | 86422 
newsapps | 93349 
BloombergMedia | 6415567 
dukechronicle | 2545549 
times | 2771911 
censusreporter | 1772387
NYTimes | 72081 
nprapps | 247435
fivethirtyeight | 3176682
TimeMagazine | 2886039

<i> source: https://github.com/collections/open-journalism </i>

GitHub Collection: <b>Getting_started_with_machine_learning</b> is the collection of open source machine learning library and repositories of open source datasets

name | org_id
--- | --- 
scikit-learn | 23655 
aikorea | 8557654 
Theano | 9504 
GSA | 63708 
nationalparkservice | 637539 
tenserflow | 10451648 
deepmind | 6170681 
GoogleTrends | 8026505
apache | 13369 
shogun-toolbox | 41562
src-d | 10092617
openai | 10741300
fivethirtyeight | 2886039

<i> source: https://github.com/collections/machine-learning </i>

GitHub Collection: <b>Hacking Minecraft</b> is the collection of projects related to Minecraft game.

name | org_id
--- | --- 
Bukkit | 9938 
docker | 234594 
MovingBlocks | 4660 
overviewer | 9321 
essentials | 19347 
PrismarineJS | 7115233 
MinecraftForge | 9635 
SpigotMC | 1983947
msmhq | 8742236 
PocketMine | 1463194
minefold | 60842
MightyPirates | 3084255
GlowstoneMC | 5035267
cuberite | 6485533

<i> source: https://github.com/collections/hacking-minecraft </i>

Below, we import the dataframe contains 'id' and 'username' of these all organizations.

In [105]:
%sh wget -P/FileStore/tables https://www.dropbox.com/s/rq1x8e2e9cv63sa/openjournalism.csv

In [106]:
%sh wget -P/FileStore/tables https://www.dropbox.com/s/8wed5cv656zd51o/machinelearning.csv

In [107]:
%sh wget -P/FileStore/tables https://www.dropbox.com/s/wx8q18sfqcgtci5/minecraft.csv

In [108]:
machineLearning = spark.read.csv('file:/FileStore/tables/machinelearning.csv', header=True)
openJournalism = spark.read.csv('file:/FileStore/tables/openjournalism.csv', header=True)
minecraft = spark.read.csv('file:/FileStore/tables/minecraft.csv', header=True)

In [109]:
#Here I am only examining machine learning and minecreaft datasets
#combining machine learning and minecreaft datasets
collection = unionAll(machineLearning, minecraft)
collection.createOrReplaceTempView('collections')

In [110]:
%sql
SELECT 
  A.org_id, C.login, A.user_id, F.follower_id
FROM
  organizations AS A JOIN followers AS F ON A.user_id = F.user_id JOIN collections AS C ON A.org_id = C.id
WHERE A.org_id IN (SELECT id FROM collections)

In [111]:
#spark dataframe for above stated query
CF_col = spark.sql(
'''
SELECT 
  A.org_id, C.login, A.user_id, F.follower_id
FROM
  organizations AS A JOIN followers AS F ON A.user_id = F.user_id JOIN collections AS C ON A.org_id = C.id
WHERE A.org_id IN (SELECT id FROM collections)
'''
)

#dataframe to RDD in form of tuples like: (organization, coMember)
rdd_CF_col = CF_col.rdd.map(lambda p: (str(p[1]), str(p[3])))

#convert flat key value list to list of value for each key
groupRDD_CF_col = rdd_CF_col.groupByKey().mapValues(list)

#removing dublicate values if any
groupRDD_CF_col_ = groupRDD_CF_col.map(lambda p: (p[0],list(set(p[1]))))

#RDD back to Dataframe
DF_CF_col = spark.createDataFrame(groupRDD_CF_col_ , ['organizations', 'features'])

#vectorizing the dataset
DF2_CF_col = vectorizer(DF_CF_col, 'features', 'vectors', 2000, 5)

#dimension reduction of dataset
DF3_CF_col = getPCA(DF2_CF_col, 'vectors', 2)

#### 6.6.1 Visualization using Matplotlib

In [113]:
#x and y values of data in list form
X_colCF = DF3_CF_col.rdd.map(lambda p: p['pcaFeatures'][0]).collect()
Y_colCF = DF3_CF_col.rdd.map(lambda p: p['pcaFeatures'][1]).collect()

#organization names in a list
colCF = DF3_CF_col.rdd.map(lambda p: p[0]).collect()

In [114]:
fig6, axs6 = plt.subplots()
plt.title("Co-Following on machine-learning organizations and minecraft game add-ons companies")
axs6.scatter(X_colCF,Y_colCF)
for i, txt in enumerate(colCF):
    axs6.annotate(txt, (X_colCF[i],Y_colCF[i]))
fig6.set_size_inches(18.5, 12, forward=True)
display()

####6.6.2 Plot Interpretation
+ This graph shows significant result among others.
+ `essentials`, `GlowstoneMC`, `SpigotMC`, `Bukkit`, `MinecraftForge`, `PocketMine`, `overviewer`, `MightyPirates` are all clustered together as they all have minecraft add-on projects. While machine learning organizations such as `scikit-learn`, `tensorflow`, `theano` grouped at the bottom.

##7. Interpretation of Result
+ Notes on Classification Result
  + As co-following and co-membership data get very sparse for less number of members per organization. Therefore, we deliberately selected 3 organization who has average 600 members in each of them. 
  + To make the classification run quickly, we selected the vector size of 1000 and (minDF=5) *(see the function 'vectorizer' for more detail)*
  + The average classification accuracy showed almost similar result in co-membership and co-following. Moreover, We are not limiting the co-following data or co-membership data by any means as the author has done in main paper for twitter. We used the exhaustive list of co-following and co-members.
  + The accuracy is not measured for general sample due to time constraint of project.
+ Comparison of Classification and PCA Result
  + Both co-membership and co-followings features given nearly similar classification accuracy for 3 organization selected. However, co-membership shown relatively high accuracy than co-following which suggests that co-membership would be prefered choice over co-following for PCA analysis.
  + On the contrary, PCA showed more interpretable result with co-following than co-membership which might be due to fact that co-following relation is distributed evenly across all user but organization-user membership relation is not evenly distributed. This is because of some of the organizations do not allow members' list to be viewed publically.

##8. Conclusions:
+ Co-following similarity is more significant than coMembership similarity since the membership doesn't reflect the interest of the user because user him/herself cannot join any organization as a member. Moreover, organizations in GitHub can decide whether they want their memebrs to be publicly visible or not. Which makes the dataset not very accurate.
+ While followship is totally a user's call. That's why following relationship shows more significant result in similarity measurement.
+ Top organizations are not necessarily be similar to each other and it is reflected in the result here. However, the curated list (aka Collections) of github repositories shows more similar-to-real-world result of similarity.
+ Since an organization could have more than one repositories and each being very different to each other. The members involved in certain project attracts certain user group as following and others have very different followings which may result in less significant similarity result.
+ Organizations similarity needed to be cross validated which will require organizations' business insight and how organizations differs in term of their tendency to use propriotery software architectures and frameworks as well as their OpenSource business philosophy and agendas. This kind of in-depth knowledge might help to interpret the result more meaningfully. However, it get restricted by scope and time limit of project.  

##9. Future Work:
+ I would like to extend this project to check the co-following nature in Repositories as well. Because  users can also watch or fork any open repository of their choice. This users' watching and forking other repositories might results into finding more similar repositories which can be recommended to similar users.
+ Moreover, repositories similarity could be evaluated in terms of their programming language, main objective of project and trending topics.

##10. REFERENCES
1. Venkata Rama Kiran Garimella and Ingmar Weber. 2014. Co-following on twitter. In Proceedings of the 25th ACM conference on Hypertext and social media (HT '14). ACM, New York, NY, USA, 249-254. DOI=http://dx.doi.org/10.1145/2631775.2631820
2. Gousios, G. (2013). The GHTorrent dataset and tool suite. In Proceedings of the 10th Working Conference on Mining Software Repositories (pp. 233–236). Retrieved from http://www.gousios.gr/bibliography/G13.html
3. GHimpact. http://www.gh-impact.com/about/
4. GitHub Collecitons : https://github.com/collections
5. Georgios Gousios and Diomidis Spinellis. 2012. GHTorrent: GitHub's data from a firehose. In Proceedings of the 9th IEEE Working Conference on Mining Software Repositories (MSR '12). IEEE Press, Piscataway, NJ, USA, 12-21.
6. J.   Davis   and   M.   Goadrich, “The   relationship between   Precision-Recall   and   ROC   curves,”in Proceedings of the 23rd international conference on Machine learning  -ICML ’06, 2006, pp. 233–240.
7. E. Kalliamvakou, G. Gousios, and K. Blincoe, “The Promises and Perils of Mining GitHub,”Proc. 11th, pp. 92–101, 2014.